In [1]:
import findspark
findspark.init('/tmp/spark-3.0.0-preview2-bin-hadoop2.7')

import pyspark
import random

In [2]:
from pyspark.sql import SparkSession

In [3]:
from pyspark import SparkContext
#from pyspark import SparkConf
from pyspark.sql import SQLContext
from pyspark.sql.types import *

In [4]:
spark = SparkSession.builder.appName('Basics').getOrCreate()

/tmp/spark-3.0.0-preview2-bin-hadoop2.7/python/pyspark/context.py:219: DeprecationWarning: Support for Python 2 and Python 3 prior to version 3.6 is deprecated as of Spark 3.0. See also the plan for dropping Python 2 support at https://spark.apache.org/news/plan-for-dropping-python-2-support.html.
  DeprecationWarning)


In [5]:
sc = spark.sparkContext

In [7]:
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

In [10]:
cs_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('ConferenceSeries.csv')

In [30]:
ci_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('ConferenceInstances.csv')

In [13]:
affiliations_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('Affiliations2.csv')

In [14]:
paper_df = sqlContext.read.format('com.databricks.spark.csv').options(header='true', inferschema='true').load('Paper2.csv')

In [15]:
cs_df.show(4)

+----------+------+----------------+--------------------+------------+----------+
|      CSID|CSRank|CSNormalizedName|       CSDisplayName|CSPaperCount|CSCitation|
+----------+------+----------------+--------------------+------------+----------+
|1141689323| 11674|            IEEM|Industrial Engine...|        7218|      7851|
|2727030889| 13563| BigDataSecurity|International Con...|         120|       122|
|2757782454| 25000|            SSVM|Scale Space and V...|           1|        21|
|2754909024| 13978|           ICARM|International Con...|         115|        31|
+----------+------+----------------+--------------------+------------+----------+
only showing top 4 rows



In [31]:
ci_df.show(4)

+--------------+------+--------------+-------------+----------+-------------------+--------------------+----------+----------+---------------+---------------+-------------------+-------------------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|  CIDisplayNm|    CICSID|           Location|       CIOfficialURL| StartDate|   EndDate|AbstractRegDate|SubDeadlineDate|NotificationDueDate|FinalVersionDueDate|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-------------+----------+-------------------+--------------------+----------+----------+---------------+---------------+-------------------+-------------------+------------+------------------+
|    2312404054| 25000| siggraph 2016|SIGGRAPH 2016|1164321581|Anaheim, California|http://s2016.sigg...|2016-07-24|2016-07-28|           null|     2016-01-18|               null|         2016-01-19|         846|              1741|
|     121793801| 25000|     wsdm 2012|    WSDM 2012|1120384002|        Seatt

In [20]:
from pyspark.sql.functions import isnan,when,count,col

In [32]:
ci_df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ci_df.columns]).show()

+--------------+------+--------------+-----------+------+--------+-------------+---------+-------+---------------+---------------+-------------------+-------------------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|CIDisplayNm|CICSID|Location|CIOfficialURL|StartDate|EndDate|AbstractRegDate|SubDeadlineDate|NotificationDueDate|FinalVersionDueDate|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-----------+------+--------+-------------+---------+-------+---------------+---------------+-------------------+-------------------+------------+------------------+
|             0|     0|             0|          0|     0|      14|         4302|     2549|   2548|          11754|           4471|               7607|               8577|        3674|              3674|
+--------------+------+--------------+-----------+------+--------+-------------+---------+-------+---------------+---------------+-------------------+-------------------+------------+-----

In [33]:
dropcols = ["CIOfficialURL","StartDate","EndDate","AbstractRegDate","SubDeadlineDate","NotificationDueDate","FinalVersionDueDate"]

In [34]:
ci_df= ci_df.drop(*dropcols)

In [35]:
ci_df.show(5)

+--------------+------+--------------+-------------+----------+--------------------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|  CIDisplayNm|    CICSID|            Location|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-------------+----------+--------------------+------------+------------------+
|    2312404054| 25000| siggraph 2016|SIGGRAPH 2016|1164321581| Anaheim, California|         846|              1741|
|     121793801| 25000|     wsdm 2012|    WSDM 2012|1120384002|         Seattle, WA|          87|              3496|
|    2625789444| 25000|     iprm 2006|    IPRM 2006|2621445615|  Princeton, NJ, USA|        null|              null|
|      82574969| 25000|     foga 2009|    FOGA 2009|1159491267|    Orlando, FL, USA|          19|               382|
|     196500321| 25000|     kmis 2013|    KMIS 2013|1158252963|Vilamoura, Algarv...|          32|                12|
+--------------+------+--------------+-------------+----------+-

In [36]:
ci_df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ci_df.columns]).show()

+--------------+------+--------------+-----------+------+--------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|CIDisplayNm|CICSID|Location|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-----------+------+--------+------------+------------------+
|             0|     0|             0|          0|     0|      14|        3674|              3674|
+--------------+------+--------------+-----------+------+--------+------------+------------------+



In [39]:
ci_df = ci_df.filter(((ci_df["CIPaperCount"] != "") | (ci_df["CIPaperCount"].isNotNull()))&((ci_df["CIPaperCitationCnt"] != "") | (ci_df["CIPaperCitationCnt"].isNotNull())))

In [40]:
ci_df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ci_df.columns]).show()

+--------------+------+--------------+-----------+------+--------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|CIDisplayNm|CICSID|Location|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-----------+------+--------+------------+------------------+
|             0|     0|             0|          0|     0|       1|           0|                 0|
+--------------+------+--------------+-----------+------+--------+------------+------------------+



In [41]:
ci_df = ci_df.filter(((ci_df["Location"] != "") | (ci_df["Location"].isNotNull())))

In [42]:
ci_df.show(4)

+--------------+------+--------------+-------------+----------+--------------------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|  CIDisplayNm|    CICSID|            Location|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-------------+----------+--------------------+------------+------------------+
|    2312404054| 25000| siggraph 2016|SIGGRAPH 2016|1164321581| Anaheim, California|         846|              1741|
|     121793801| 25000|     wsdm 2012|    WSDM 2012|1120384002|         Seattle, WA|          87|              3496|
|      82574969| 25000|     foga 2009|    FOGA 2009|1159491267|    Orlando, FL, USA|          19|               382|
|     196500321| 25000|     kmis 2013|    KMIS 2013|1158252963|Vilamoura, Algarv...|          32|                12|
+--------------+------+--------------+-------------+----------+--------------------+------------+------------------+
only showing top 4 rows



In [43]:
ci_df.select([count(when(isnan(c) | col(c).isNull(),c)).alias(c) for c in ci_df.columns]).show()

+--------------+------+--------------+-----------+------+--------+------------+------------------+
|ConfInstanceID|CIRank|CINormalizedNm|CIDisplayNm|CICSID|Location|CIPaperCount|CIPaperCitationCnt|
+--------------+------+--------------+-----------+------+--------+------------+------------------+
|             0|     0|             0|          0|     0|       0|           0|                 0|
+--------------+------+--------------+-----------+------+--------+------------+------------------+



In [45]:
cics_join = ci_df.join(cs_df, ci_df.CICSID == cs_df.CSID)
cics_join.show(10)

+--------------+------+--------------+-------------+----------+--------------------+------------+------------------+----------+------+----------------+--------------------+------------+----------+
|ConfInstanceID|CIRank|CINormalizedNm|  CIDisplayNm|    CICSID|            Location|CIPaperCount|CIPaperCitationCnt|      CSID|CSRank|CSNormalizedName|       CSDisplayName|CSPaperCount|CSCitation|
+--------------+------+--------------+-------------+----------+--------------------+------------+------------------+----------+------+----------------+--------------------+------------+----------+
|    2312404054| 25000| siggraph 2016|SIGGRAPH 2016|1164321581| Anaheim, California|         846|              1741|1164321581|  8426|        SIGGRAPH|International Con...|       15838|    468071|
|     121793801| 25000|     wsdm 2012|    WSDM 2012|1120384002|         Seattle, WA|          87|              3496|1120384002| 10055|            WSDM|Web Search and Da...|         826|     26832|
|      82574969

In [46]:
cics_join.count()

9954

In [47]:
paper_df.show()

+----------+-----+--------------------+----------+--------------------+--------------------+---------+----------+--------------------+----------+------------+--------------+--------------+-------------+--------------+
|   PaperID| Rank|                 DOI|   DocType|          PaperTitle|       OriginalTitle|PaperYear| PaperDate|           Publisher| JournalID|ConfSeriesID|ConfInstanceID|ReferenceCount|CitationCount|EstimatedCount|
+----------+-----+--------------------+----------+--------------------+--------------------+---------+----------+--------------------+----------+------------+--------------+--------------+-------------+--------------+
|1819143627|24199|10.1109/PICMET.20...|Conference|technological inn...|Technological inn...|     2001|2001-01-01|                IEEE|  23074404|  2734834919|    2735348883|             0|            0|             0|
|1963578506|22929|10.1109/ICIEEM.20...|Conference|a hybrid due date...|A hybrid due-date...|     2010|2010-10-01|               